# Search image in the Dataset

With this notebook you can search for photos using natural language.

## Load the dataset

You will need the Unsplash Dataset and the precomputed feature vetors for this. You don't want to process the whole dataset yourself, you can download the preprocessed feature vectors from [here](https://drive.google.com/drive/folders/1ozUUr8UQ2YWDSJyYIIun9V8Qg1TjqXEs?usp=sharing).

In [7]:
from pathlib import Path
import numpy as np
import pandas as pd

# Set the paths
features_path = Path(r"C:\Code\visual-search-engine-contest\data\features")


# Load the features and the corresponding IDs
photo_features = np.load(features_path / "features.npy")
photo_ids = pd.read_csv(features_path / "photo_ids.csv")
photo_ids = list(photo_ids['photo_id'])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Code\\visual-search-engine-contest\\data\\features\\features.npy'

Load the CLIP network.

In [ ]:
from CLIP import clip
import torch

# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

## Search

Specify your search query and encode it to a feature vector using CLIP.

In [ ]:
search_query = "David Ben Gurion with a child"

with torch.no_grad():
    # Encode and normalize the description using CLIP
    text_encoded = model.encode_text(clip.tokenize(search_query).to(device))
    text_encoded /= text_encoded.norm(dim=-1, keepdim=True)

Compare the text features to the image features and find the best match.

In [3]:
# Retrieve the description vector and the photo vectors
text_features = text_encoded.cpu().numpy()

# Compute the similarity between the descrption and each photo using the Cosine similarity
similarities = list((text_features @ photo_features.T).squeeze(0))

# Sort the photos by their similarity score
best_photos = sorted(zip(similarities, range(photo_features.shape[0])), key=lambda x: x[0], reverse=True)

NameError: name 'text_encoded' is not defined

## Display the results

Show the top 3 results.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

# Iterate over the top 3 results
for i in range(4):
    # Retrieve the photo ID
    idx = best_photos[i][1]
    photo_id = photo_ids[idx]


    # Display the photo
    display(Image(f"C:\\Users\\elady\\Documents\\NLI\\images\\raw_data\\unzipped\\{photo_id}\\{photo_id}.jpg"))
    print()
    